# Using a pre-trained NLP model for Question-Answering

In this notebook, we look at an example for fine-tuning a pre-trained NLP model for a question answering (QA) task. We'll use the Hugging Face transformers library and a pre-trained BERT model. The task involves improving the model's ability to answer questions based on a given context.

## Without Fine-Tuning
First, let's see how the pre-trained BERT model performs on a QA task without any fine-tuning. We'll use an example where the model needs to find an answer within a given passage.

In [58]:
from transformers import BertTokenizer, BertForQuestionAnswering, AutoTokenizer, DefaultDataCollator, TrainingArguments, Trainer
import torch

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForQuestionAnswering: ['distilbert.transformer.layer.4.attention.k_lin.weight', 'distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.4.ffn.lin1.bias', 'vocab_projector.bias', 'distilbert.transformer.layer.5.attention.q_lin.bias', 'distilbert.transformer.layer.2.ffn.lin2.weight', 'distilbert.transformer.layer.2.attention.q_lin.bias', 'distilbert.transformer.layer.4.output_layer_norm.bias', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.transformer.layer.1.ffn.lin2.bias', 'distilbert.transformer.layer.1.attention.k_lin.bias', 'distilbert.transformer.layer.3.sa_layer_norm.weight', 'distilbert.transformer.layer.5.ffn.lin1.bias', 'distilbert.tran

Define a context and a question where the model might initially struggle

In [59]:
context = "The University of California was founded in 1868, located in Berkeley."
question = "When was the University of California established?"

#### Model Prediction
Tokenize the input, make a prediction, and decode the answer:

In [60]:
inputs = tokenizer(question, context, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs)

# Find the tokens with the highest `start` and `end` scores
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1

# Convert tokens to answer string
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0, answer_start:answer_end]))
print("Answer:", answer)

Answer: [CLS] when was the university of california established? [SEP] the university of california was founded in 1868, located


## With Fine-Tuning

Now, let's fine-tune this BERT model on a similar task to potentially improve its performance. For this task we will use the Stanford Question Answering Dataset (SQuAD) dataset, which is a large-scale dataset designed to test the reading comprehension ability of machine learning models. It contains 100,000+ question - answers created by humans on a range of Wikipedia articles, similar to the below example.

<img src="./imgs/squad_example.webp" alt="drawing" width="450"/>

### Load SQuAD dataset

In [43]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:100]")
squad = squad.train_test_split(test_size=0.2)


Let's take a look at an example:

In [19]:
squad["train"][0]

{'id': '5733a6424776f41900660f52',
 'title': 'University_of_Notre_Dame',
 'context': 'The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study – aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering – with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A. and Master of Business Administration (MBA) degrees, respectively.',
 'question': 'The College of Science began to offer civil engineering courses beginning at what time at Notre Dame?',
 'answers': {'text': ['the 1870s'], 'answer_start': [155]}}

There important fields here:

- `answers`: the starting location of the answer token and the answer text.
- `context`: background information from which the model needs to extract the answer.
- `question`: the question a model should answer.

#### Preprocessing
There are a few preprocessing steps particular to question answering tasks you should be aware of:

- Some examples in a dataset may have a very long context that exceeds the maximum input length of the model. To deal with longer sequences, truncate only the context by setting truncation="only_second".
- Next, map the start and end positions of the answer to the original context by setting `return_offset_mapping=True.`
- With the mapping in hand, now you can find the start and end tokens of the answer. Use the `sequence_ids` method to find which part of the offset corresponds to the question and which corresponds to the context.

To apply the preprocessing function over the entire dataset, `map` function. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once.

In [44]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=128,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_squad = squad.map(preprocess_function, batched=True)
data_collator = DefaultDataCollator()

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

### Train
We can now finetune the model

In [66]:

training_args = TrainingArguments(
    output_dir="qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Reduced to 1 epoch for demonstration
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.9023065567016602, 'eval_runtime': 3.8615, 'eval_samples_per_second': 5.179, 'eval_steps_per_second': 0.518, 'epoch': 1.0}
{'train_runtime': 167.0232, 'train_samples_per_second': 0.479, 'train_steps_per_second': 0.03, 'train_loss': 0.4272572994232178, 'epoch': 1.0}


TrainOutput(global_step=5, training_loss=0.4272572994232178, metrics={'train_runtime': 167.0232, 'train_samples_per_second': 0.479, 'train_steps_per_second': 0.03, 'train_loss': 0.4272572994232178, 'epoch': 1.0})

### Evaluate

We have seen how even a single epoch of fine-tuning can refine the model's understanding and improve the answering accuracy. Fine-tuning can potentially improve the model's accuracy significantly depending on the nature and amount of the fine-tuning data

In [67]:
context = "The University of California was founded in 1868, located in Berkeley."
question = "When was the University of California established?"

# Tokenize the context to find the exact start and end position of the answer
encoded = tokenizer.encode_plus(question, context, return_tensors="pt")
input_ids = encoded["input_ids"].tolist()[0]

model.eval()
with torch.no_grad():
    outputs = model(**encoded)

answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1

# Convert tokens to answer string
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print("Improved Answer:", answer)

Improved Answer: 1868


We have seen how even a single epoch of fine-tuning can refine the model's understanding and improve the answering accuracy. Fine-tuning can potentially improve the model's accuracy significantly depending on the nature and amount of the fine-tuning data. In practice, we would consider more robust training process such as  increasing training examples, epochs and the `max_length` parameter. 